<a href="https://colab.research.google.com/github/martinlee192/AI-basic/blob/master/ResNet34_x_ray_cnn_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)
print(device)

cuda


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
!dir /gdrive/MyDrive/data

Mounted at /gdrive
model_simple_layer_20epoch.pth	x_ray_test_data      x_ray_train_data
model_vgg16_50epoch.pth		x_ray_test_data.zip


In [ ]:
#nomalization 값 구해서 넣기!!!!!!!!!
trans = transforms.Compose([
    transforms.ToTensor(),
     ])

trainset = torchvision.datasets.ImageFolder(root='/gdrive/MyDrive/data/x_ray_train_data', transform=trans)

In [ ]:
import numpy as np

meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in trainset]
stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in trainset]

meanR = np.mean([m[0] for m in meanRGB])
meanG = np.mean([m[1] for m in meanRGB])
meanB = np.mean([m[2] for m in meanRGB])

stdR = np.mean([s[0] for s in stdRGB])
stdG = np.mean([s[1] for s in stdRGB])
stdB = np.mean([s[2] for s in stdRGB])

print(meanR, meanG, meanB)
print(stdR, stdG, stdB)

0.48235524 0.48235524 0.48235524
0.22150697 0.22150697 0.22150697


캐글에서 다른 사람의 경우
[0.485, 0.456, 0.406] , [0.229,0.224,0.225] 사용

resnet50 예시
https://www.kaggle.com/code/teyang/pneumonia-detection-resnets-pytorch/notebook

resnet34 도 있다
https://www.kaggle.com/code/vishnurapps/identifying-pneumonia-from-xrays-using-resnet

normalization 구하기 https://deep-learning-study.tistory.com/475

In [ ]:
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  [0.229,0.224,0.225])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  [0.229,0.224,0.225])
])

trainset = torchvision.datasets.ImageFolder(root='/gdrive/MyDrive/data/x_ray_train_data', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=10,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.ImageFolder(root='/gdrive/MyDrive/data/x_ray_test_data', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=10,
                                         shuffle=False, num_workers=0)

classes = ('Normal','PNEUMONIA')


In [ ]:
#''' 이렇게 해도 된다.
#import resnet
#conv1x1=resnet.conv1x1
#Bottleneck = resnet.Bottleneck
#BasicBlock= resnet.BasicBlock
#'''

In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

1. BasicBlock for Resnet18,34
3x3 cov, c = 64
3x3 cov, c = 64
'+'
relu

2. Bottleneck for Resnet 50,101,152
1x1, 64
3x3, 64
1x1, 256(64*expension)
'+' relu
3.'+'는 이전 입력을 더해 주는 걸 뜻하며 Identity라고 단어 사용

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride) # 받아온 stride를 넣어줌
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)      # 따로 안넣었으니 stride =1 이다
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):

        identity = x

        out = self.conv1(x) # 3x3 stride = 2
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out) # 3x3 stride = 1
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes) #conv1x1(64,64)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)#conv3x3(64,64)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion) #conv1x1(64,256)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x) # 1x1 stride = 1
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out) # 3x3 stride = stride 
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out) # 1x1 stride = 1
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [ ]:
class ResNet(nn.Module):
    # model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs) #resnet 50 
    def __init__(self, block, layers, num_classes=2, zero_init_residual=True):
        super(ResNet, self).__init__()
        #shape = [ 3 x 224 x 224]
        self.inplanes = 64
             
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        #shape = [ 64 x 112 x 112] 
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        #shape = [ 64 x 112 x 112] 
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        #shape = [ 64 x 56 x 56] 
        self.layer1 = self._make_layer(block, 64, layers[0])#'''3'''
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)#'''4'''
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)#'''6'''
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)#'''3'''
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)
    
    def _make_layer(self, block, planes, blocks, stride=1):
        
        downsample = None
        
        if stride != 1 or self.inplanes != planes * block.expansion: 
            
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride), #conv1x1(256, 512, 2)
                nn.BatchNorm2d(planes * block.expansion), #batchnrom2d(512)
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        
        self.inplanes = planes * block.expansion #self.inplanes = 128 * 4
        
        for _ in range(1, blocks): 
            layers.append(block(self.inplanes, planes)) # * 3

        return nn.Sequential(*layers)
    

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
def resnet18(pretrained=False, **kwargs):
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs) #=> 2*(2+2+2+2) +1(conv1) +1(fc)  = 16 +2 =resnet 18
    return model

In [ ]:
def resnet34(pretrained= False, **kwargs):
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs) #=> 2*(3+4+6+3) +1(conv1) +1(fc) = 32 + 2 = resnet 34
    return model


In [ ]:
def resnet50(pretrained=False, **kwargs):
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs) #=> 3*(3+4+6+3) +(conv1) +1(fc) = 48 +2 = 50
    return model

In [ ]:
model_resnet34 = resnet34().to(device)

In [ ]:
a=torch.Tensor(1,3,224,224).to(device)
out = model_resnet34(a)
print(out)
out.shape

tensor([[-0.0177, -0.6249]], device='cuda:0', grad_fn=<AddmmBackward0>)


torch.Size([1, 2])

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model_resnet34.parameters(), lr = 0.1, momentum = 0.9, weight_decay=5e-4)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

define acc_check function

In [ ]:
def acc_check(net, test_set, epoch, save=0):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_set:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)

            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc = (100 * correct / total)
    print('Accuracy of the network on the %d test images: %d %%' %total , acc)
    if save:
        torch.save(net.state_dict(), "./model/model_epoch_{}_acc_{}.pth".format(epoch, int(acc)))
    return acc

In [ ]:
print(len(trainloader))
total_batch = len(trainloader)
epochs = 150

for epoch in range(epochs):  # loop over the dataset multiple times

    avg_cost = 0.0
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model_resnet34(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        avg_cost += loss / total_batch
    
    lr_sche.step()
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
    if epochs%50 == 49:
          torch.save(model_resnet34.state_dict(), "/gdrive/MyDrive/data/Resnet34_epoch_{}.pth".format(epoch))

print('Finished Training')

522
[Epoch:1] cost = 0.23021014034748077
[Epoch:2] cost = 0.16510172188282013
[Epoch:3] cost = 0.14413471519947052
[Epoch:4] cost = 0.131316140294075
[Epoch:5] cost = 0.1263149231672287
[Epoch:6] cost = 0.10945085436105728
[Epoch:7] cost = 0.11652301251888275
[Epoch:8] cost = 0.1108417958021164
[Epoch:9] cost = 0.1105998307466507
[Epoch:10] cost = 0.06523022800683975
[Epoch:11] cost = 0.06825044751167297
[Epoch:12] cost = 0.06100349500775337
[Epoch:13] cost = 0.06032335013151169
[Epoch:14] cost = 0.06575620919466019
[Epoch:15] cost = 0.061594750732183456
[Epoch:16] cost = 0.062108442187309265
[Epoch:17] cost = 0.0681547150015831
[Epoch:18] cost = 0.05649698153138161
[Epoch:19] cost = 0.06316456943750381
[Epoch:20] cost = 0.029857849702239037
[Epoch:21] cost = 0.01995094306766987
[Epoch:22] cost = 0.01783050037920475
[Epoch:23] cost = 0.02360515482723713
[Epoch:24] cost = 0.026058875024318695
[Epoch:25] cost = 0.026781458407640457
[Epoch:26] cost = 0.019061468541622162
[Epoch:27] cost =

In [ ]:
torch.save(model_resnet34.state_dict(),"/gdrive/MyDrive/data/model_ResNet34_150epoch.pth")

In [ ]:
TestM_resnet34 = resnet34().to(device)
TestM_resnet34.load_state_dict(torch.load('/gdrive/MyDrive/data/model_ResNet34_150epoch.pth'))

<All keys matched successfully>

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = TestM_resnet34(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d test images: %d %%' % (total,
    100 * correct / total))

Accuracy of the network on the 624 test images: 75 %
